In [1]:
import comtypes.client as cc
import PyPDF2
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
import os
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import Button, Checkbox, VBox, Output, IntProgress
from IPython.display import display, clear_output

In [2]:
def choose_files_from_directory(optional_argument=None):
    root = Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_paths = askopenfilenames(title="Select Files", filetypes=(
        ("All files", "*.*"), ("PDF files", "*.pdf")))
    root.destroy()
    return file_paths

# Convert Word document to PDF


def convert_word_to_pdf(word_file, pdf_file):
    try:
        word = cc.CreateObject("Word.Application")
        doc = word.Documents.Open(os.path.abspath(word_file))
        doc.SaveAs(os.path.abspath(pdf_file), FileFormat=17)
        doc.Close()
        word.Quit()
    except Exception as e:
        print("Error converting Word document to PDF:", str(e))
    finally:
        if word is not None:
            word.Quit()

# Convert Excel sheet to PDF


def convert_excel_to_pdf(excel_file, pdf_file):
    try:
        excel = cc.CreateObject("Excel.Application")
        workbook = excel.Workbooks.Open(os.path.abspath(excel_file))
        workbook.ExportAsFixedFormat(0, os.path.abspath(pdf_file))
        workbook.Close()
        excel.Quit()
    except Exception as e:
        print("Error converting Excel sheet to PDF:", str(e))
    finally:
        if excel is not None:
            excel.Quit()


def combine_pdfs(input_files, output_file):
    pdf_writer = PyPDF2.PdfWriter()

    for file in input_files:
        with open(file, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                pdf_writer.add_page(page)

    with open(output_file, 'wb') as output:
        pdf_writer.write(output)


def convert_pdf(selected_files, ranking, combine):
    total_steps = len(selected_files)
    progress = IntProgress(
        value=0,
        min=0,
        max=total_steps,
        description='Progress:',
        bar_style='success',
        style={'bar_color': 'green'}
    )
    display(progress)

    word_files = [x for x in selected_files if any(
        keyword.lower() in x.lower() for keyword in ['.docx', '.doc'])]
    excel_files = [x for x in selected_files if any(
        keyword.lower() in x.lower() for keyword in ['.xls', '.xlsx', '.csv', '.xlsm'])]
    pdf_files = [x for x in selected_files if any(
        keyword.lower() in x.lower() for keyword in ['.pdf'])]

    if len(word_files + excel_files + pdf_files) != len(selected_files):
        print(
            "Incompatible document detected! Please choose from list of supported documents")
        return None
    # Convert Word document to PDF
    for word_file in word_files:
        file = word_file.rfind(".")
        pdf_file = word_file[:file] + ".pdf"
        print(f"Converting {word_file}")
        convert_word_to_pdf(word_file, pdf_file)
        print(f"Converted {pdf_file} succesfully.")
        progress.value += 1
        # pdf_files.append(pdf_file)

    # Convert Excel sheet to PDF
    for excel_file in excel_files:
        file = excel_file.rfind(".")
        pdf_file = excel_file[:file] + ".pdf"
        print(f"Converting {excel_file}")
        convert_excel_to_pdf(excel_file, pdf_file)
        print(f"Converted {pdf_file} succesfully.")
        progress.value += 1
        # pdf_files.append(pdf_file)

    # Combine PDFs
    if combine:
        pdf_path = [os.path.abspath(x) for x in ranking]
        output_file = download_dir = os.path.join(os.path.expanduser(
            "~"), "Downloads") + '\combined' + datetime.now().strftime("%Y%m%d_%H%M%S") + '.pdf'
        print(f"Combining all PDFs to {output_file}")
        combine_pdfs(pdf_path, output_file)
        print(f"Combined all PDFs to {output_file}")
        print("Completed! Check Downloads folder for combined PDF.")
        progress.value = len(selected_files)

In [3]:
output = widgets.Output()


@output.capture()
def select_files(start_button):
    global dropdowns, selected_files
    clear_output()
    selected_files = choose_files_from_directory()
    get_files_button = [x.split("/")[-1] for x in selected_files]

    # Create a dropdown widget for each item
    dropdowns = [widgets.Dropdown(options=get_files_button, value=item, description=f'File {i+1}:')
                 for i, item in enumerate(get_files_button)]

    # Create a VBox container to hold the dropdowns
    container = widgets.VBox(dropdowns)

    if len(selected_files) > 0:
        display(container, combine_checkbox, button)
    else:
        print("Please select at least one file.")

# Define a function to get the ranked list when the button is clicked


def start_conversion(button):
    with output:
        ranked_list = []
        rank_file_name = [dropdown.value for dropdown in dropdowns]
        for filename in rank_file_name:
            for filename_original in selected_files:
                if filename in filename_original:
                    ranked_list.append(
                        filename_original[:filename_original.rfind(".")] + ".pdf")
        convert_pdf(selected_files, ranked_list,
                    combine=combine_checkbox.value)


# Create a button to display the user's ranked list
start_button = widgets.Button(description='Select Documents')
button = widgets.Button(description='Convert')
combine_checkbox = widgets.Checkbox(description='Combine')

# Register the button click event
button.on_click(start_conversion)
start_button.on_click(select_files)
# Display the dropdowns and button
display(start_button, output)

Button(description='Select Documents', style=ButtonStyle())

Output()